In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
diabetes = pd.read_csv('C:/IDE/data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [6]:
# Задание 8.1
dupl_columns = list(diabetes.columns)
diabetes_dedupped = diabetes.drop_duplicates(subset=dupl_columns)
print(diabetes_dedupped.shape[0])

768


In [9]:
# задание 8.2
low_information_cols = []

for col in diabetes_dedupped.columns:
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    nunique_ratio = diabetes_dedupped[col].nunique()/diabetes_dedupped[col].count()
    if top_freq > 0.95:
        low_information_cols.append(col)
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
      
print(low_information_cols)  
hight_inf_diabetes = diabetes_dedupped.drop(low_information_cols, axis=1)
hight_inf_diabetes.head()          

['Gender']


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98,58,33,190,34.0,0.430,43,0
1,2,112,75,32,0,35.7,0.148,21,0
2,2,108,64,0,0,30.8,0.158,21,0
3,8,107,80,0,0,24.6,0.856,34,0
4,7,136,90,0,0,29.9,0.210,50,0


In [48]:
# Задание 8.3
feature_list = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for feature in hight_inf_diabetes:
    if feature in feature_list:
        hight_inf_diabetes[feature] = hight_inf_diabetes[feature].replace(0, np.nan)
                
cols_null_percent = round(hight_inf_diabetes.isnull().mean().sort_values(ascending=False), 2)
display(cols_null_percent)   

Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
BMI                         0.01
Glucose                     0.01
Pregnancies                 0.00
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

In [49]:
# Задание 8.4
clean_diabet_df = hight_inf_diabetes.copy()

thresh = clean_diabet_df.shape[0] * 0.7
clean_diabet_df = clean_diabet_df.dropna(thresh=thresh, axis=1)
display(clean_diabet_df.shape[1])

8

In [50]:
# Задание 8.5
m = clean_diabet_df.shape[1]
clean_diabet_df = clean_diabet_df.dropna(thresh=m-2, axis=0)
print(clean_diabet_df.shape[0])

761


In [52]:
clean_diabet_df.isnull().mean()

Pregnancies                 0.000000
Glucose                     0.006570
BloodPressure               0.036794
SkinThickness               0.289093
BMI                         0.005256
DiabetesPedigreeFunction    0.000000
Age                         0.000000
Outcome                     0.000000
dtype: float64

In [54]:
# Задание 8.6
values = {
    'Glucose': clean_diabet_df['Glucose'].median(),
    'BloodPressure': clean_diabet_df['BloodPressure'].median(),
    'SkinThickness': clean_diabet_df['SkinThickness'].median(),
    'BMI': clean_diabet_df['BMI'].median()
}
clean_diabet_df = clean_diabet_df.fillna(values)
round(clean_diabet_df['SkinThickness'].mean(), 1)

29.1

In [55]:
# Задание 8.7
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (1.5*iqr)
    upper_bound = quartile_3 + (1.5*iqr)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaners = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaners

outliers, cleaners = outliers_iqr(clean_diabet_df, 'SkinThickness')
print(f'Число выбросов: {outliers.shape[0]}')

Число выбросов: 87


In [56]:
# Задание 8.8
def outliers_z(data, feature):
    x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - (3*sigma)
    upper_bound = mu + (3*sigma)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaners = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaners

outliers, cleaners = outliers_z(clean_diabet_df, 'SkinThickness')
print(f'Число выбросов: {outliers.shape[0]}')

Число выбросов: 4


In [58]:
# Задание 8.9
def outliers_iqr_not_log(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (1.5*iqr)
    upper_bound = quartile_3 + (1.5*iqr)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaners = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaners

outliers, cleaners = outliers_iqr_not_log(clean_diabet_df, 'DiabetesPedigreeFunction')
print(f'Число выбросов: {outliers.shape[0]}')


def outliers_iqr_log(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (1.5*iqr)
    upper_bound = quartile_3 + (1.5*iqr)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaners = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaners

outliers, cleaners = outliers_iqr_log(clean_diabet_df, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов: {outliers.shape[0]}')        

Число выбросов: 29
Число выбросов: 0
